## Requirements
On importe toutes les librairies nécéssaires

In [5]:
import numpy as np
import re
import pandas as pd
#!pip install unidecode
from unidecode import unidecode
import torch
import ast

In [6]:
from googlesearch import search

In [7]:
from PyPDF2 import PdfReader

In [8]:
import os
pd.set_option('mode.chained_assignment', None)
import s3fs

In [9]:
import matplotlib.pyplot as plt

## Fonctions

In [10]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

In [11]:
def normalize_text(s):
    """removing caractères, /, accents, m3, /n"""
    import string, re
    
    def no_unit(text):
        regex="m3"
        return re.sub(regex, " ", text)
    def no_exp(text):
        regex="m³"
        return re.sub(regex, " ", text)
    def no_punct(text):
        regex="\\.(?!\d)"
        return re.sub(regex," ",text)
   
    def white_space_fix(text):
        return " ".join(text.split())
    def no_date(text):
        date_extract_pattern = "[0-9]{1,2}\\/[0-9]{1,2}\\/[0-9]{4}"
        return re.sub(date_extract_pattern,"",text)
    def special_char(text):
        regex=regex = "[^\w\s,.]"
        return re.sub(regex, " ", text)
   #on retire tout les caractère des réponses 
    def no_char(text):
        regex=r"([a-zA-Z])"
        return re.sub(regex, " ", text)
    def comma(text):
        regex=",(?!\d)"
        return re.sub(regex," ",text)
    def point(text):
        return re.sub("\,",".",text)
    def indic(text):
        regex="(P\d\d\d\\.\d\s*)|(D\d\d\d\\.\d\s*)|(VP\\.\d\d\d)"
        return re.sub(regex," ", text)

    return white_space_fix(point(no_punct(no_char(unidecode(comma(special_char(indic(no_exp(no_unit(no_date(s)))))))))))

In [12]:
def normalize(s):
    """removing caractères, /, accents, m3, /n"""
    import string, re
    def no_punct(text):
        regex="\\.(?!\d)"
        return re.sub(regex," ",text)
    def white_space_fix(text):
        return " ".join(text.split())
    def special_char(text):
        regex=regex = "[^\w\s,.]"
        return re.sub(regex, " ", text)
    def comma(text):
        regex=",(?!\d)"
        return re.sub(regex," ",text)
    def lower(text):
        return text.lower()

    return white_space_fix(no_punct(unidecode(lower(special_char(s)))))

In [13]:
def normalize_an(s):
    """removing caractères, /, accents, m3, /n"""
    import string, re
    
    def no_unit(text):
        regex="m3"
        return re.sub(regex, " ", text)
    def no_exp(text):
        regex="m³"
        return re.sub(regex, " ", text)
    def no_punct(text):
        regex="\\.(?!\d)"
        return re.sub(regex," ",text)
   
    def white_space_fix(text):
        return "".join(text.split())
    def no_date(text):
        date_extract_pattern = "[0-9]{1,2}\\/[0-9]{1,2}\\/[0-9]{4}"
        return re.sub(date_extract_pattern,"",text)
    def special_char(text):
        regex=regex = "[^\w\s,.]"
        return re.sub(regex, " ", text)
   #on retire tout les caractère des réponses 
    def no_char(text):
        regex=r"([a-zA-Z])"
        return re.sub(regex, " ", text)
    def comma(text):
        regex=",(?!\d)"
        return re.sub(regex," ",text)
    def point(text):
        return re.sub("\,",".",text)
    def indic(text):
        regex="(P\d\d\d\\.\d\s*)|(D\d\d\d\\.\d\s*)|(VP\\.\d\d\d)"
        return re.sub(regex," ", text)

    return white_space_fix(point(no_punct(no_char(unidecode(comma(special_char(indic(no_exp(no_unit(no_date(s)))))))))))

In [14]:
def white_space_fix(text):
        return " ".join(text.split())

In [15]:
def lower(text):
        return text.lower()

In [16]:
def saut(text):
        regex="\n"
        return re.sub(regex,"",text)

In [17]:
def compute_f1(prediction, truth):
    pred_tokens =normalize(prediction).split()
    truth_tokens = normalize(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    #print(len(common_tokens))
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

## Charger les modèles
On charge le modèle de question réponse d'etalab pour appliquer les pré-filtres, puis le modèle T5 pour récupérer les valeurs d'indicateurs.

In [18]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
from transformers import pipeline
nlp = pipeline('question-answering', model='etalab-ia/camembert-base-squadFR-fquad-piaf', tokenizer='etalab-ia/camembert-base-squadFR-fquad-piaf')

## Charger les données de SISPEA
On charge le documents contenant toutes les questions: associées à un mot clé et à un indicateur

### Charger les questions

In [20]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "/mberthe/narval"
FILE_KEY_Q = "camemBERT/question_all.csv"
FILE_PATH_Q = BUCKET + "/" + FILE_KEY_Q

with fs.open(FILE_PATH_Q,  mode="r") as file_in:
    question=pd.read_csv(file_in)

In [21]:
question

,question,mot,indic
0,Quel est le prix de service au m3 pour l'eau p...,prix,D102.0
1,Quel est le prix du service au m3 pour l'assin...,prix,D204.0
2,Quelle est la connaissance du réseau en eau po...,connaissance,P103.2B
3,Quel est le taux de connaissance du réseau en ...,connaissance,P202.2B
4,Quel est le taux de renouvellement des réseaux...,renouvellement,P107.2
...,...,...,...
97,Quelle est la valeur de l'endettement du servi...,dette,P256.2
98,Quelle est la valeur de l'endettement du servi...,endettement,P256.2
99,Quelle est la valeur de l'endettement du servi...,dette,P153.2
100,Quel est le total de points de connaissance d...,connaissance,P202.2B


## Pré-filtres
On récupère les premieres pages du rapport, desquelles on extrait l'année du rapport aisni que le type de service proposé 

In [22]:
#  Fonction extrait l'intro
def extract_intro(pdf_path):
    response = requests.get(pdf_path)
    with io.BytesIO(response.content) as f:
        reader = PdfReader(f)
        page = reader.pages
        text = []
        for i in range(0, 5):
            page = reader.pages[i]
            output = page.extract_text()
            text.append(output)
            text1 = ''
        for x in text:
            text1 += ' ' + x
    return text1

In [41]:
def prefiltres(text1):
    global df, df_ville
    df_ville=pd.DataFrame()
    # année
    année = nlp({'question': "Quelle est l'année d'exercice du rapport ?", 
    'context': text1})
    an = normalize_an(année['answer'])
    print(an)

    # service
    reau= "eau potable"
    rass="assainissement"

    d=nlp({'question': "Quels types de services ? ",
         'context': text1})
    print(d)
    prediction=d["answer"]
    prediction=normalize(prediction)
    if ((re.findall(reau,prediction)==[])&(re.findall(rass,prediction)==[])):
        d=nlp({'question': "eau potable ou assainissement collectif ? ",
         'context': text1})
        print(d)
        prediction=d["answer"]
        prediction=normalize(prediction)
        if ((re.findall(reau,prediction)==[])&(re.findall(rass,prediction)==[])):
            d=nlp({'question': "Quelle est l'entité de gestion ? ",
             'context': text1})
            print(d)
            prediction=d["answer"]
            prediction=normalize(prediction)
            if ((re.findall(reau,prediction)==[])&(re.findall(rass,prediction)==[])):
                print("type de service non retrouvé dans le texte")
    truth="""eau potable"""
    prediction=d['answer']
    print(prediction)
    f1_ep=compute_f1(prediction,truth)
    print('f1 eau potable:',f1_ep)

    truth="""assainissement"""
    prediction=normalize(d['answer'])
    f1_ac=compute_f1(prediction,truth)
    print('f1 assainissement:',f1_ac)

    if ((re.findall(reau,prediction)!=[])&(re.findall(rass,prediction)!=[])):
        print(re.findall(reau,prediction),re.findall(rass,prediction))
        indic=["D102.0","D204.0","P103.2","P202.2","P107.2","P253.2","P109.0","P207.0","P153.2","P256.2","P154.0","P257.0","P155.1","P258.1","P101.1","P102.1","P104.3","P105.3","P106.3","P108.3","P151.1","D203.0","P201.1","P204.3","P205.3","P206.3","P251.1","P252.2","P254.3","P255.3","D302.0","P301.3"]
        service="AC+EP"
    else:
        if (f1_ep > f1_ac ):
            indic=["D102.0","P103.2B","P107.2","P109.0","P153.2","P154.0","P155.1","P101.1","P102.1","P104.3","P105.3","P106.3","P108.3","P151.1"]
            service="EP"
        if (f1_ep < f1_ac):
            indic=["D204.0","P202.2B","P253.2","P207.0","P256.2","P257.0","P258.1","D203.0","P201.1","P204.3","P205.3","P206.3","P251.1","P252.2","P254.3","P255.3","D302.0","P301.3"]
            service="AC"
    if(f1_ep==0)&(f1_ac==0):
             indic=["D102.0","D204.0","P103.2B","P202.2B","P107.2","P253.2","P109.0","P207.0","P153.2","P256.2","P154.0","P257.0","P155.1","P258.1","P101.1","P102.1","P104.3","P105.3","P106.3","P108.3","P151.1","D203.0","P201.1","P204.3","P205.3","P206.3","P251.1","P252.2","P254.3","P255.3","D302.0","P301.3"]
             service="Not defined"
    print(indic)
    print(service)
    
    #ville
    d_ville=nlp({'question': "Quel est le nom de la ville?",
         'context': text1})
    ville=d_ville['answer']
    ville=normalize(ville)
    ville=normalize(ville)
    print(ville)
        #on charge le csv associé à l'année et au service
    try:
        FILE_KEY_Q = "camemBERT" + "/" + "data_"+an +"_"+ service +".xlsx" #Nom du fichier chargé depuis sispea
        print(FILE_KEY_Q)
        FILE_PATH_Q = BUCKET + "/" + FILE_KEY_Q
        with fs.open(FILE_PATH_Q,  mode="rb") as file_in:
           df = pd.read_excel(file_in)
        for i in df.index:
            df["Nom collectivité"][i]=lower(df["Nom collectivité"][i])
        cols=df.columns
        df_ville=pd.DataFrame(columns=[cols])
        f1=0
        row_list=[]
        ind=''
        for i in df.index:
            if normalize(df["Nom collectivité"][i])==normalize(ville):
                  print('ville trouvée',i,df["Nom collectivité"][i],ville)
                  row_list.append(df.loc[i])
        df_ville=pd.DataFrame(row_list)
        for i in df.index:
            #print(type(f1))
                if compute_f1(df["Nom collectivité"][i],ville) > f1:
                    print(i)
                    ind=i 
                    f1=compute_f1(df["Nom collectivité"][i],ville)
        if not ind:
            print('ville not found')
        else:
                row_list.append(df.loc[ind])
                df_ville=pd.DataFrame(row_list)
                print(df["Nom collectivité"][ind])
                print(ind)

    
    except (FileNotFoundError):
        print("fichier de données n'est pas dans s3")
        df_ville.empty
    if df_ville.empty:
        ('empty2')

    return(indic,service,an,df_ville,ville)

In [42]:
        FILE_KEY_Q = "camemBERT/data_2020_AC.xlsx" #Nom du fichier chargé depuis sispea
        print(FILE_KEY_Q)
        S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
        fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
        BUCKET = "/mberthe/narval"  
        FILE_PATH_Q = BUCKET + "/" + FILE_KEY_Q
        with fs.open(FILE_PATH_Q,  mode="rb") as file_in:
           df = pd.read_excel(file_in)

camemBERT/data_2020_AC.xlsx


FileNotFoundError: /mberthe/narval/camemBERT/data_2020_AC.xlsx

## On extrait tout le pdf

In [25]:
#  Fonction extrait le texte entier
def extract_text(pdf_path):
    response = requests.get(pdf_path)
    with io.BytesIO(response.content) as f:
        reader = PdfReader(f)
        page=reader.pages
        count = len(reader.pages)
        print(count)
        text=[]
        for i in range(count):
            page = reader.pages[i]
            output = page.extract_text()
            text.append(output)
    return(text)

In [26]:
def search(text):
    #Recherche mot clés
    pages =[]
    for i in question.index:
        my_list=[]
        if question["indic"][i] in indic:
            for j in text:
                #print(re.findall(f1["mot"][i],j))
                    if (re.findall(question["mot"][i],j)!=[]):
                        my_list.append(j)
                        #print("question:",i,"page",text.index(j))
            #print(my_list)
        pages.append(my_list)
    return(pages)

## Prédictions

In [27]:
   #Prédictions
def predictions(pages):
    global re
    pred=[]
    for i in question.index:
        #print("question :",i)
        list_pred=[]
        for j in pages[i]:
            #print("page:",text.index(j))
            input_text = 'contexte: '+ j + 'question: ' + question['question'][i] + "en "+ an + " ?"
            input_ids = tokenizer(input_text, max_length=300,
                truncation="only_second",
                padding="max_length",return_tensors="pt").input_ids.to('cuda')
            outputs = model.generate(input_ids)
            list_pred.append(tokenizer.decode(outputs[0]))
            #print(tokenizer.decode(outputs[0]))
        pred.append(list_pred)
   
    return(pred)

In [28]:
def mef_pred(pred):
    global re,fs
     #mise en forme pred
    pred1=question
    pred1["pred"]=pred
    for i in pred1.index :
        pred1["pred"][i]=list(map(normalize_text, pred1["pred"][i]))
    #on affiche que les réponses prédites et pas les vides
    for i in pred1.index:
        if not (pred1["pred"][i]):
            pred1=pred1.drop([i])
    table=pd.DataFrame()
    for i in indic:
        my_list=[]
        for j in pred1.index:
            if (pred1["indic"][j]==i):
                my_list=my_list + pred1["pred"][j]
        #print(i,my_list)
        d={"indic":i,"valeur":my_list}
        #print(d)
        df_dictionary = pd.DataFrame([d])
        table = pd.concat([table, df_dictionary], ignore_index=True)
    print(table.head())
    table.head()

    for l in table.index:
        for i in table['valeur'][l]:
            if (re.findall('\d+',i)==[]):
                table['valeur'][l].remove(i)
    for l in table.index:
        for i in table['valeur'][l]:        
            if (i==''):
                table['valeur'][l].remove(i)
    for l in table.index:
        for i in table["valeur"][l]:
            i = [item.split(' ') for item in i]

    #save and load pred
    BUCKET_OUT = "mberthe/narval"
    FILE_KEY_OUT_S3 = "camemBERT/pre_" + ville + service + an + "_T5_xl.csv" #modif nom du csv
    FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

    with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
        table.to_csv(file_out,index=False)
                    
    S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
    fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
    BUCKET = "mberthe/narval"
    FILE_KEY_S3 ="camemBERT/pre_" + ville + service + an + "_T5_xl.csv" #modif nom du csv 
    FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3
    print("save pre")
    with fs.open(FILE_PATH_S3, mode="rb",index= False) as file_in:
        pred = pd.read_csv(file_in)

    #split list     
    for i in pred.index:
       pred['valeur'][i] =res = ast.literal_eval(pred['valeur'][i])
    for i in pred.index:
        pred['valeur'][i]= [item.split(' ') for item in pred['valeur'][i]]
    for l in pred.index:
        temp=[]
        for i in pred['valeur'][l]:
            for j in i:
                print(j)
                temp.append(j)
        pred['valeur'][l]=temp


    import re
    for l in pred.index:
         my_list=[]
         for i in pred['valeur'][l]:
           if (re.findall('\d+',i)==[]):
                my_list.append(i)
         for i in my_list:
             pred['valeur'][l].remove(i)
    
            
    #Entiers
    reg=r"\d\.(?!0)"
    for i in pred.index:
        if pred["indic"][i] in ["P103.2","P202.2","P254.3","P255.3"]:
            print(pred['valeur'][i])
            temp=[] 
            for x in pred['valeur'][i]:
                print(x)
                if (re.findall(reg,x)!=[]):
                    temp.append(x)
                    print('ajoute x à temp:', x)
            for x in temp:
                pred['valeur'][i].remove(x)
                print('retire :',x)
            print(pred['valeur'][i])

    for i in pred.index:
        pred['valeur'][i]=[float(x) for x in pred['valeur'][i] if isfloat(x)]
    for i in pred.index:
        pred["valeur"][i]=set(pred["valeur"][i])
    

    #bornes
    S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
    fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
    BUCKET = "mberthe/narval"
    FILE_KEY_S3 = "camemBERT/bornes.csv"
    FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

    with fs.open(FILE_PATH_S3, mode="rb",index=False) as file_in:
        bornes = pd.read_csv(file_in)
    pred['min']=""
    pred['max']=""
    for i in bornes.index:
        for j in pred.index:
            if bornes['indic'][i]==pred['indic'][j]:
                pred["min"][j]=float(bornes['min'][i])
                pred['max'][j]=float(bornes['max'][i])
    for i in pred.index:
        temp=[]
        for x in pred['valeur'][i]:
            print(pred['indic'][i],x,type(x),pred['min'][i],type(pred['min'][i]))
            if (float(x)<pred['min'][i]) or (float(x)>pred['max'][i]):
                temp.append(x)
        for x in temp:
            pred['valeur'][i].remove(x)
    
    #RF
    BUCKET_OUT = "mberthe/narval"
    FILE_KEY_OUT_S3 = "camemBERT/rf_" + ville + service + an + "_T5_xl.csv"   #modif nom csv
    FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3
    print("save rf")
    with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
        pred.to_csv(file_out,index=False)
    return(pred)

## Comparaison avec sispea

In [4]:
def comp_sisp(pred_f,df_ville):
    global fs
    S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
    fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
    BUCKET = "mberthe/narval"
    FILE_KEY_S3 = "camemBERT/results.csv"
    FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

    with fs.open(FILE_PATH_S3, mode="rb",index=False) as file_in:
        results = pd.read_csv(file_in, sep= ';')

    S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
    fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
    BUCKET = "mberthe/narval"
    FILE_KEY_S3 = "camemBERT/indic_metric.csv"
    FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

    with fs.open(FILE_PATH_S3, mode="rb",index=False) as file_in:
        indic_metric = pd.read_csv(file_in, sep= ';')
    
    in_list=0
    exact=0
    for k in df_ville.index:
        print(k)
        in_list_temp=0
        exact_temp=0
        true_n=0
        false_p=0
        false_n=0
        for i in pred.index:
            for col in df_ville.columns:
            #for j in df_ville.index:
                    if pred["indic"][i]==col:
                        df_ville[col][k]=round(df_ville[col][k],2)
                        print(col,'pred:',pred['valeur'][i],'true:',df_ville[col][k])
                    #Le nombre de bonnes réponses dans la liste des réponses prédites 
                        if df_ville[col][k] in pred["valeur"][i]:
                            in_list_temp=in_list_temp+1
                            print("in_list +1")
                            for indic in indic_metric.index:
                                if indic_metric["indic"][indic]==col:
                                    indic_metric["in_list"][indic]=indic_metric["in_list"][indic] +1
                    #Le nombre de réponses exactes (true positive)    
                        if {df_ville[col][k]}== pred["valeur"][i]:
                            exact_temp=exact_temp+1
                            print("exact +1")
                            for indic in indic_metric.index:
                                if indic_metric["indic"][indic]==col:
                                    indic_metric["exact"][indic]=indic_metric["exact"][indic] +1
                    #Le nombre de true negative (= pas de réponses retournée car pas de réponses dans le PDF)    
                        if np.isnan(df_ville[col][k]) & (pred['valeur'][i]== set()):
                            true_n=true_n+1
                            print('true_n +1')
                    #Le nombre de false positive(=Il y a une préiction alors qu'il n'y a pas de réponse dans le PDF)
                        if np.isnan(df_ville[col][k]) & (pred['valeur'][i]!= set()):
                            false_p= false_p+1
                            print('false_p +1')    
                    #Le nombre de false negative(=Il n'y a pas de predition alors qu'il y a une réponse dans le PDF)
                        if (not (np.isnan(df_ville[col][k]))) & (pred['valeur'][i]== set()):
                            false_n= false_n+1
                            print('false_n +1')    
                    
        print('code postal:',k,in_list_temp,exact_temp)
        if in_list_temp>in_list:
            in_list=in_list_temp
            exact=exact_temp
    print('in_list:',in_list)
    print('exact(=true_pos):',exact)
    print('true_neg: ',true_n)
    print('false_neg:',false_n)
    print('false_pos:',false_p)

    BUCKET_OUT = "mberthe/narval"
    FILE_KEY_OUT_S3 = "camemBERT/indic_metric.csv"#modif nom csv
    FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

    with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
        indic_metric.to_csv(file_out,index=False,sep=';')
    
    #acc
    accuracy=(exact+true_n)/len(pred)
    accuracy=round(accuracy,2)
    print(accuracy)

    #recall
    if (exact+false_n)!=0:
        recall=(exact)/(exact+false_n)
    else: 
        recall=0
    recall=round(recall,2)
    recall

    #precision
    if (exact+false_p)!=0:
        precision=(exact)/(exact+false_p)
    else:
        precision=0
    precision=round(precision,2)
    precision

    #f1
    if (recall+precision)!=0:
        f1=2*(recall*precision)/(recall+precision)
    else: 
        f1=0
    f1=round(f1,2)
   
    print(in_list,exact)

    d_result={'nom_fichier':nom_fichier,'service':service,'in_list':in_list,'exact':exact,'true_neg':true_n,'false_pos':false_p,'false_neg':false_n,'recall':recall,'precision':precision,'accuracy':accuracy,'f1':f1,'len':count}
    df_result=pd.Series(d_result)
    df_result=df_result.to_frame()
    df_result=df_result.transpose()
    results=pd.concat([results,df_result])
    BUCKET_OUT = "mberthe/narval"
    FILE_KEY_OUT_S3 = "camemBERT/results.csv"#modif nom csv
    FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

    with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
        results.to_csv(file_out,index=False,sep=';')
        return(results)

## main

In [43]:
from googlesearch import search
requête='RPQS'
links=[]

for url in  search(requête,tld="fr",lang='fr',num=10,start=0,stop=None,pause=2.0):
    print("URL: ",url)
    try:
        read = requests.get(url)
 
# full html content
        html_content = read.content
 
# Parse the html content
        soup = BeautifulSoup(html_content, "html.parser")
        for link in soup.find_all('a'):
            current_link = link.get('href')
        #print(current_link)
            if current_link!=None:
                if current_link.endswith('pdf'):
                    print("PDF_link",current_link)
                    links.append(current_link)
    except Exception:
        pass
links

URL:  https://www.seine-et-marne.gouv.fr/Actions-de-l-Etat/Environnement-et-cadre-de-vie/Eau/Observatoire-national-des-services-publics-d-eau-et-d-assainissement/Le-Rapport-sur-le-Prix-et-la-Qualite-du-Service-RPQS
URL:  https://www.services.eaufrance.fr/gestion/rpqs
URL:  https://www.eure.gouv.fr/Actions-de-l-Etat/Environnement/eau-et-nature/Eau/Services-publics-d-eau-potable-et-d-assainissement/Le-rapport-sur-le-prix-et-la-qualite-des-services-RPQS-d-eau-et-d-assainissement
URL:  https://www.eure-et-loir.gouv.fr/Politiques-publiques/Environnement/Les-services-publics-d-eau-potable-et-d-assainissement/Le-rapport-sur-le-prix-et-la-qualite-du-service-RPQS
URL:  https://www.haute-loire.gouv.fr/le-rapport-sur-le-prix-et-la-qualite-des-services-r733.html
PDF_link http://www.services.eaufrance.fr/docs/plaquettes-observatoire/Fiche-teleRPQS_HD-DEF.pdf
URL:  https://www.aube.gouv.fr/Politiques-publiques/Environnement/Eau/Politique-de-l-eau/Observatoire-des-services-publics-d-eau-et-d-assainis

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


URL:  https://www.haute-savoie.gouv.fr/Actions-de-l-Etat/Votre-departement/Eau-et-assainissement/Outils-et-modeles-pour-les-services-publics-d-eau-et-d-assainissement
URL:  https://www.gers.gouv.fr/Actions-de-l-Etat/Environnement/Gestion-de-l-eau/Observatoire-national-des-services-publics-d-eau-et-d-assainissement-SISPEA
URL:  https://www.sidealf.fr/rpqs-eau-potable-annee-2022
PDF_link /sites/default/files/2023-06/rpqs_eau_potable_2022.pdf
URL:  https://scot-region-grenoble.org/transitions/wp-content/uploadsFTP/sispea%20-%20ddt%201.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


URL:  https://www.leshautesrivieres.com/articles/rapport-annuel-sur-le-prix-et-la-qualite-du-service-public-rpqs-de-l-assainissement-collectif-2018
URL:  https://smdea09.fr/docutheque/le-rapport-sur-le-prix-et-la-qualite-du-service-rpqs-2020/
PDF_link https://smdea09.fr/wp-content/uploads/2022/01/RPQS__SMDEA__2020-_compresse.pdf
PDF_link https://smdea09.fr/wp-content/uploads/2021/12/deliberation_n°2424_RPQS_2020.pdf
PDF_link https://smdea09.fr/wp-content/uploads/2021/02/RPQS__SMDEA__2019.pdf
PDF_link https://smdea09.fr/wp-content/uploads/2020/11/rpqs__smdea__2018_vd.pdf
URL:  https://www.eau17.fr/publications/rpqs-eau-potable-2021
URL:  https://www.smemac.fr/rapport-annuel
URL:  https://www.ypresia.fr/prestations/bilans-rapports/automatisation-du-rpqs/
URL:  https://sivom-regionminiere.fr/pp-rapport-prix-et-qualita-des-services.html
PDF_link http://www.sivom-regionminiere.fr/upload/telechargements/RPQS_2014.pdf
PDF_link http://www.sivom-regionminiere.fr/upload/telechargements/RPQS_2015

...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


URL:  https://www.siebag.fr/le-rpqs-2013/
PDF_link https://www.siebag.fr/wp-content/uploads/2014/07/RPQS-AEP-2013.pdf
URL:  https://www.terredes2caps.fr/rapports-sur-la-qualite-et-le-prix-du-service
PDF_link /sites/default/files/2022-01/sgs20210922-q19_annexe_rpqs_eau_potable_2020.pdf
PDF_link /sites/default/files/2022-01/rpqs_anc_2020_210721_0.pdf
PDF_link /sites/default/files/2022-01/rpqs_ac_2020_210721.pdf
URL:  https://www.info-flash.com/info-flash/actualites/france/auvergne-rhone-alpes/ain/305-belley/58148226-agent-agente-recherche-de-fuites-eau-potable-h-f.html
URL:  https://pqms.cgg.gov.in/pqms-angular/login
URL:  https://ppqs.gov.in/
URL:  https://www.emploi-environnement.com/offre/communaute-communes-ouche-montagne-chargee-mission-etudes-eau-assainissement-285065.html
URL:  https://choisirleservicepublic.gouv.fr/offre-emploi/agent-e-recherche-de-fuites-eau-potable---communaute-de-com-bugey-sud-reference-O001230701112150/
PDF_link https://choisirleservicepublic.gouv.fr/wp-conte

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


URL:  http://xn--80adbmhl0ai6b6e.xn--p1ai/201621.html
URL:  https://www.indiabix.com/verbal-ability/ordering-of-words/
URL:  http://comdoctor.ru/743292.html
URL:  https://www.ign.com/lists/top-100-rpgs
URL:  http://inspiredwalldecor.com/287359.html
URL:  http://lccnavi.net/?wptouch_switch=desktop&redirect=http%3A//bacbgrg.work/ZqXpil26lccnavinetttp3
URL:  http://yashkinocrb.ru/307632.html
URL:  http://tube2017.com/?mod=count&id=676&url=http%3A//r3590.cfd/IOlpil26tube2017comttp3
URL:  https://www.denofgeek.com/games/best-rpgs-ever-console-pc/
URL:  http://www.decoset.fr/
URL:  https://www.registre-coproprietes.gouv.fr/annuaire


['http://www.services.eaufrance.fr/docs/plaquettes-observatoire/Fiche-teleRPQS_HD-DEF.pdf',
 '/rpqs/Exemple_RPQS_AEP_2.10.pdf',
 '/rpqs/Exemple_RPQS_ASST_2.10.pdf',
 '/rpqs/Exemple_RPQS_ANC_2.10.pdf',
 'http://services.eaufrance.fr/upload/P101.1_fiche.pdf',
 'http://services.eaufrance.fr/upload/P101.1_fiche.pdf',
 '/sites/default/files/2023-06/rpqs_eau_potable_2022.pdf',
 'https://smdea09.fr/wp-content/uploads/2022/01/RPQS__SMDEA__2020-_compresse.pdf',
 'https://smdea09.fr/wp-content/uploads/2021/12/deliberation_n°2424_RPQS_2020.pdf',
 'https://smdea09.fr/wp-content/uploads/2021/02/RPQS__SMDEA__2019.pdf',
 'https://smdea09.fr/wp-content/uploads/2020/11/rpqs__smdea__2018_vd.pdf',
 'http://www.sivom-regionminiere.fr/upload/telechargements/RPQS_2014.pdf',
 'http://www.sivom-regionminiere.fr/upload/telechargements/RPQS_2015.pdf',
 'http://www.sivom-regionminiere.fr/upload/telechargements/RPQS_2016.pdf',
 'http://www.sivom-regionminiere.fr/upload/telechargements/RPQS_2016.pdf',
 'http://www

In [36]:
import requests
from bs4 import BeautifulSoup
import io
from PyPDF2 import PdfFileReader

In [44]:
for i in range(0,20):
    try:
        print(i)
        text1=extract_intro(links[i])
        print("intro extract")
        indic,service,an,df_ville,ville = prefiltres(text1)
        print('prefiltres')
        print("lien",i,"indic: ",indic,"an:", an, "df_ville",df_ville,"ville:",ville)
        text=extract_text(links[i])
        pages=search(text)
        pred=predictions(pages)
        pred_f =mef_pred(pred)
        if  not df_ville.empty:
            df_results= comp_sisp(pred_f)
    except :
        pass


0
1
2
3
4
5
6
7
intro extract
2007
{'score': 0.19692741334438324, 'start': 65, 'end': 115, 'answer': ' services publics de l’eau et de \nl’assainissement'}
 services publics de l’eau et de 
l’assainissement
f1 eau potable: 0.1818181818181818
f1 assainissement: 0.19999999999999998
['D204.0', 'P202.2B', 'P253.2', 'P207.0', 'P256.2', 'P257.0', 'P258.1', 'D203.0', 'P201.1', 'P204.3', 'P205.3', 'P206.3', 'P251.1', 'P252.2', 'P254.3', 'P255.3', 'D302.0', 'P301.3']
AC
camemBERT/data_2007_AC.xlsx
fichier de données n'est pas dans s3
couserans
prefiltres
lien 7 indic:  ['D204.0', 'P202.2B', 'P253.2', 'P207.0', 'P256.2', 'P257.0', 'P258.1', 'D203.0', 'P201.1', 'P204.3', 'P205.3', 'P206.3', 'P251.1', 'P252.2', 'P254.3', 'P255.3', 'D302.0', 'P301.3'] an: 2007 df_ville Empty DataFrame
Columns: []
Index: [] ville: couserans
137
8
9
intro extract
2018
{'score': 0.23871806263923645, 'start': 82, 'end': 116, 'answer': ' de l ’eau et de \nl’assainissement'}
 de l ’eau et de 
l’assainissement
f1 eau pota